In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from pathlib import Path

BASE_DIR = Path().resolve().parents[1]

In [ ]:
atc1_raw = pd.read_csv(BASE_DIR /"data/processed/atc1_prescriptions_2014_2024.csv", sep = ";")

In [229]:
atc1_raw.drop(columns = ["population", "boites", "region"], inplace = True, errors = "ignore")


In [230]:
atc1 = atc1_raw.groupby(["atc1", "l_atc1", "annee"])["boites_pour_1000"].sum()
atc1 = atc1.reset_index()

In [233]:
list_code = ('A', 'B', 'C', 'D', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S','V')
X_futur = [[2025], [2026], [2027], [2028], [2029], [2030]]
dict_pred = {"atc1" : "", "annee" : "", "pred_boite": ""}
df_pred = []

for atc1_code in list_code:

    X = atc1.query(f"atc1 == '{atc1_code}'")["annee"].values.reshape(-1,1)
    Y = atc1.query(f"atc1 == '{atc1_code}'")["boites_pour_1000"]

    model = LinearRegression().fit(X, Y)
    list_pred = (model.predict(X_futur).round(0).astype(int))

    count = 0
    for pred in list_pred:
        dict_pred["annee"] = X_futur[count][0]
        dict_pred["atc1"] = atc1_code
        dict_pred["pred_boite"] = pred

        df_pred.append(dict_pred)
        dict_pred = {"atc1" : "", "annee" : "", "pred_boite": ""}
        count += 1

df_pred = pd.DataFrame(df_pred)

In [234]:
atc1.drop(columns = ["annee", "boites_pour_1000"], inplace = True, errors = "ignore")
df_pred = df_pred.merge(atc1, on = ["atc1"], how = "left").drop_duplicates()

In [2]:
atc1_raw = pd.read_csv(BASE_DIR /"data/processed/atc1_prescriptions_2014_2024.csv", sep = ";")
atc1_raw.drop(columns = ["population", "boites", "region"], inplace = True, errors = "ignore")
atc1 = atc1_raw.groupby(["atc1", "l_atc1", "annee"])["boites_pour_1000"].sum()
atc1 = atc1.reset_index()

In [237]:
df_pred = df_pred[["annee", "atc1", "l_atc1", "pred_boite"]]
df_pred = pd.concat([atc1, df_pred], ignore_index=True)
df_pred["pred_boite"] = df_pred["pred_boite"].astype("Int64")
df_pred["boites_pour_1000"] = df_pred["boites_pour_1000"].astype("Int64")



In [255]:
df_pred.sample(10)

,atc1,l_atc1,annee,boites_pour_1000,pred_boite
111,P,"ANTIPARASITAIRES, INSECTICIDES ET REPULSIFS",2015,1160,<NA>
157,A,SYSTEME DIGESTIF ET METABOLISME,2028,<NA>,77192
15,B,SANG ET ORGANES HEMATOPOIETQUES,2018,22362,<NA>
8,A,SYSTEME DIGESTIF ET METABOLISME,2022,74233,<NA>
207,M,SYSTEME MUSCULO-SQUELETTIQUE,2030,<NA>,7780
90,M,SYSTEME MUSCULO-SQUELETTIQUE,2016,20949,<NA>
67,J,ANTI-INFECTIEUX (USAGE SYSTEMIQUE),2015,29070,<NA>
0,A,SYSTEME DIGESTIF ET METABOLISME,2014,67644,<NA>
60,H,"HORMONES SYSTEMIQUES, A L EXCLUSION DES HORMON...",2019,11875,<NA>
169,C,SYSTEME CARDIO-VASCULAIRE,2028,<NA>,41272


In [ ]:
df_pred.to_csv(BASE_DIR /"data/processed/df_pred.csv", sep = ";", index=False)

In [ ]:
df_pred.to_parquet(BASE_DIR /"data/processed/df_pred.parquet", index=False)